<a href="https://colab.research.google.com/github/weilainicolas/Deep-Learning/blob/master/L1_R2_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai.vision import *

In [0]:
path=untar_data(URLs.PETS)

In [0]:
path.ls()

[PosixPath('/root/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations')]

In [0]:
pat=re.compile(r'/([^/]+)_\d+.jpg$')

In [0]:
il=ImageList.from_folder(path/'images')

In [0]:
tfms=get_transforms(do_flip=True, max_rotate=10.0, max_zoom=2.)
bs, size= (64, 128)
data=(il.split_by_rand_pct(0.1, seed=42)
       .label_from_func(lambda x: pat.search(x.as_posix())[1])
       .transform(tfms, size=size, padding_mode='reflection', do_resolve=True,resize_method=ResizeMethod.SQUISH)
       .databunch(bs=bs)
       .normalize(imagenet_stats))

In [0]:
data.c

37

In [0]:
def con(ni,nf,ks,s,p): 
  return nn.Sequential(
    nn.Conv2d(ni,nf,kernel_size=(ks,ks),stride=(s,s),padding=(p,p),bias=False),
    nn.BatchNorm2d(nf),
    nn.ReLU(inplace=True))

In [0]:
class resblock(nn.Module):
  def __init__(self,nf):
    super().__init__()
    self.conv1=con(nf,nf,3,1,1)
    self.conv2=con(nf,nf,3,1,1)
    self.conv3=con(nf,nf,3,1,1)
  def forward(self,x): return x+self.conv3(self.conv2(self.conv1(x)))

In [0]:
def con_res (ni,nf,ks,s,p): 
  return nn.Sequential(
      con(ni,nf,ks,s,p),
      resblock(nf)
  )

In [0]:
model=nn.Sequential(
    con(3,64,7,2,3),#64
    nn.MaxPool2d(3),
    con_res(64,128,3,2,1), #32
    con_res(128,256,3,2,1), #16
    con_res(256,512,3,2,1), #8
    con_res(512,1024,3,2,1),#4 
    con_res(1024,512,3,2,1), #2
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.AdaptiveMaxPool2d(output_size=1),
    Flatten(),
    nn.BatchNorm1d(512),
    nn.Dropout(p=0.25),
    nn.Linear(in_features=512,out_features=512,bias=True),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=512,out_features=37, bias=True)
)

In [0]:
learn=Learner(data,model,loss_func=nn.CrossEntropyLoss(),metrics=error_rate)

In [0]:
learn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 64, 64]         9,408      True      
______________________________________________________________________
BatchNorm2d          [64, 64, 64]         128        True      
______________________________________________________________________
ReLU                 [64, 64, 64]         0          False     
______________________________________________________________________
MaxPool2d            [64, 21, 21]         0          False     
______________________________________________________________________
Conv2d               [128, 11, 11]        73,728     True      
______________________________________________________________________
BatchNorm2d          [128, 11, 11]        256        True      
______________________________________________________________________
ReLU                 [128, 11, 11]        0          False     
___________________________________________________

In [0]:
learn.fit_one_cycle(40,max_lr=0.1)

epoch,train_loss,valid_loss,error_rate,time
0,4.148990,4.046456,0.975643,01:11
1,4.206667,3.596028,0.956698,01:11
2,4.227537,6.505754,0.960758,01:10
3,3.807411,4.599601,0.943166,01:10
4,3.747682,3.571130,0.953992,01:11
5,3.997844,4.158205,0.983762,01:12
6,4.332956,11.307730,0.955345,01:12
7,4.883533,40.285625,0.967524,01:12
8,5.006528,960250.875000,0.963464,01:14
9,5.615292,4.862250,0.960758,01:12


KeyboardInterrupt: ignored

In [0]:
learn.fit_one_cycle(40,max_lr=1e-1)

epoch,train_loss,valid_loss,error_rate,time
0,2.107482,2.240356,0.641407,01:12
1,2.127806,2.266579,0.641407,01:11
2,2.130929,2.287125,0.650880,01:11
3,2.197544,2.380430,0.680650,01:12
4,2.283287,2.462782,0.694181,01:13
5,2.368485,2.686121,0.730717,01:13
6,2.470687,2.809125,0.765900,01:14
7,2.490976,3.047320,0.818674,01:13
8,2.578040,2.716820,0.756428,01:14
9,2.643822,3.084594,0.833559,01:15


KeyboardInterrupt: ignored